In [4]:
import random
import pandas as pd
from sage.graphs.graph_generators import graphs 

# generiramo grafe - definiciji cikla in drevesa, nato generiramo grafe od 2 do 12 vozlišč:

grafi = []  

def cikel(n):
    G = Graph()
    G.add_vertices(range(n))
    e = [(i, (i + 1) % n) for i in range(n)]
    G.add_edges(e)
    return G

def drevo(i):
    return [Graph(tree) for tree in graphs.trees(i)]
 
for i in range(2, 12):  
    grafi.append(cikel(i))
    grafi.extend(drevo(i))


###########################################################

def SDCTD_stevilo2(g):
    h = g.complement()

    p = MixedIntegerLinearProgram(maximization = False)
    x = p.new_variable(binary = True)
    p.set_objective(sum([x[v] for v in g]))

    for v in g.vertices():
        neighbors = g.neighbors(v)  
        p.add_constraint(x[v] + sum(x[w] for w in neighbors) >= 1)

    zaustavi = False

    for v in g.vertices():
        neighbors_complement = h.neighbors(v)  
        if neighbors_complement:  
            p.add_constraint(sum(x[w] for w in neighbors_complement) >= 1)

        else: 
            zaustavi = True
            break 

    if not zaustavi == True:
        p.solve()
        x = p.get_values(x)
        return(sum(1 for i in x.values() if i == 1))
    else:
        return None
    
############################################################ 


In [6]:
# tukaj je prikazana analiza na 1000 nakjučno izbirah:


rezultati = []

for m in range(1, 1000):
    g1 = random.choice(grafi)
    g2 = random.choice(grafi)
    # direktni produkt grafov g1 in g2
    g3 = g1.tensor_product(g2)

    # Izračun SDCTD za vsak graf
    SDCTD_stevilo_graf1 = SDCTD_stevilo2(g1)
    SDCTD_stevilo_graf2 = SDCTD_stevilo2(g2)
    SDCTD_stevilo_graf3 = SDCTD_stevilo2(g3)

    # Dodajanje rezultatov v seznam
    rezultati.append({
        'Izbira grafov': f'Izbira {m}',  # Tukaj smo popravili
        'SDCTD_graf1': SDCTD_stevilo_graf1,
        'SDCTD_graf2': SDCTD_stevilo_graf2,
        'SDCTD_direktni': SDCTD_stevilo_graf3
    })

df = pd.DataFrame(rezultati)


def preveri_neenacbo(df, faktor, oznaka):
    def neenacba(row):
        if row['SDCTD_graf1'] is None or row['SDCTD_graf2'] is None:  # izpustimo tiste grafe z SDCTD = None
            return ''
        # Preveri, če velja pogoj za trenutni faktor
        if faktor * row['SDCTD_graf1'] * row['SDCTD_graf2'] >= row['SDCTD_direktni']:
            return True
        return False

    # Dodajanje stolpca za trenutno neenačbo
    df[oznaka] = df.apply(neenacba, axis=1)

    # Prešteje število False vrednosti
    false_count = (df[oznaka] == False).sum()

    # Izračuna delež False vrednosti
    false_ratio = false_count / len(df)

    print(f"Za {oznaka}:")
    print(f"Število False v stolpcu {oznaka}: {false_count}")
    print(f"Delež False v stolpcu {oznaka}: {false_ratio:.2%}")
    print()

# Seznam neenačb za preverjanje
neenacbe = [
    (1, 'neenacba1'),
    (2, 'neenacba2'),
    (3, 'neenacba3'),
    (4, 'neenacba4'),
    (5, 'neenacba5')
]


for faktor, oznaka in neenacbe:
    preveri_neenacbo(df, faktor, oznaka)


Za neenacba1:
Število False v stolpcu neenacba1: 769
Delež False v stolpcu neenacba1: 76.98%

Za neenacba2:
Število False v stolpcu neenacba2: 100
Delež False v stolpcu neenacba2: 10.01%

Za neenacba3:
Število False v stolpcu neenacba3: 0
Delež False v stolpcu neenacba3: 0.00%

Za neenacba4:
Število False v stolpcu neenacba4: 0
Delež False v stolpcu neenacba4: 0.00%

Za neenacba5:
Število False v stolpcu neenacba5: 0
Delež False v stolpcu neenacba5: 0.00%



In [5]:
# hitrejša analiza, izvedena na 100 nakjlučno izbranih parov grafov, hitrejše od prej, ko smo izvedli na 1000 izbirah; tukaj se izpiše še primer tabele:

rezultati = []

for m in range(1, 101):
    g1 = random.choice(grafi)
    g2 = random.choice(grafi)
    # direktni produkt grafov g1 in g2
    g3 = g1.tensor_product(g2)

    # Izračun dominacijskega števila za vsak graf
    dominacijsko_stevilo_graf1 = SDCTD_stevilo2(g1)
    dominacijsko_stevilo_graf2 = SDCTD_stevilo2(g2)
    dominacijsko_stevilo_graf3 = SDCTD_stevilo2(g3)

    # Dodajanje rezultatov v seznam
    rezultati.append({
        'Izbira grafov': f'Izbira {m}',  # Tukaj smo popravili
        'SDCTD_graf1': dominacijsko_stevilo_graf1,
        'SDCTD_graf2': dominacijsko_stevilo_graf2,
        'SDCTD_direktni': dominacijsko_stevilo_graf3
    })

df = pd.DataFrame(rezultati)


def preveri_neenacbo(df, faktor, oznaka):
    def neenacba(row):
        if row['SDCTD_graf1'] is None or row['SDCTD_graf2'] is None:  
            return ''
        # Preveri, če velja pogoj za trenutni faktor
        if faktor * row['SDCTD_graf1'] * row['SDCTD_graf2'] >= row['SDCTD_direktni']:
            return True
        return False

    # Dodajanje stolpca za trenutno neenačbo
    df[oznaka] = df.apply(neenacba, axis=1)

    # Prešteje število False vrednosti
    false_count = (df[oznaka] == False).sum()

    # Izračuna delež False vrednosti
    false_ratio = false_count / len(df)

    print(f"Za {oznaka}:")
    print(f"Število False v stolpcu {oznaka}: {false_count}")
    print(f"Delež False v stolpcu {oznaka}: {false_ratio:.2%}")
    print()

# Seznam neenačb za preverjanje
neenacbe = [
    (1, 'neenacba1'),
    (2, 'neenacba2'),
    (3, 'neenacba3'),
    (4, 'neenacba4'),
    (5, 'neenacba5')
]

# Zagon preverjanja za vsako neenačbo
for faktor, oznaka in neenacbe:
    preveri_neenacbo(df, faktor, oznaka)


# Prikaz tabele
pd.set_option('display.width', None)  

print(df.to_string(index=False))

Za neenacba1:
Število False v stolpcu neenacba1: 77
Delež False v stolpcu neenacba1: 77.00%

Za neenacba2:
Število False v stolpcu neenacba2: 7
Delež False v stolpcu neenacba2: 7.00%

Za neenacba3:
Število False v stolpcu neenacba3: 0
Delež False v stolpcu neenacba3: 0.00%

Za neenacba4:
Število False v stolpcu neenacba4: 0
Delež False v stolpcu neenacba4: 0.00%

Za neenacba5:
Število False v stolpcu neenacba5: 0
Delež False v stolpcu neenacba5: 0.00%

Izbira grafov SDCTD_graf1 SDCTD_graf2 SDCTD_direktni neenacba1 neenacba2 neenacba3 neenacba4 neenacba5
     Izbira 1        None           4              9                                                  
     Izbira 2           4           4             18     False      True      True      True      True
     Izbira 3           3           4             19     False      True      True      True      True
     Izbira 4           3        None              7                                                  
     Izbira 5           4   